<a href="https://colab.research.google.com/github/Kushagra2000/Neural_machine_translation/blob/main/Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
#imports
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
import pandas as pd
import numpy as np
import re
import unicodedata

1.15.2


In [69]:
#Upload the deu.txt file and put it in the Data Folder
!ls Data/

deu.txt


In [70]:
#reading lines from thge txt file
with open('Data/deu.txt','r') as f:
  lines=(f.readlines())

In [71]:
#removing the unnecessary data and obtaining sentence pairs 
def process_text(lines):
  ''' 
  Takes a list of tab seperated
  English and German sentences as input 
  and returns dataframe of processed
  English and German sentence pairs
  '''
  proc_lines=[]
  for line in lines:
    line=line.strip() #to remove newlines at the end of a sentence
    line=line.split("\t") #splitting by tabs
    line=line[:-1]  #remove contributing information
    
    line[0]=unicodedata.normalize('NFD',line[0])
    line[0]=line[0].lower()
    line[0]=re.sub(r"([.,?!])",r" \1 ",line[0])   #adding spaces before and after punctuation
    line[0]=re.sub(r"[0-9]",r" ",line[0])
    line[0]=re.sub(r'["]',r" ",line[0])
    line[0]=re.sub(r"[']",r" ",line[0])
    line[0]=re.sub(r"[:]",r" ",line[0])
    line[0]=re.sub(r"[ ]+"," ",line[0])  #removing excess spaces
    line[0]=line[0].strip() #removing spaces from the end of string
    line[0]="<SOS> "+line[0]+" <EOS>"

    line[1]=unicodedata.normalize('NFD', line[1])
    line[1]=line[1].lower()
    line[1]=re.sub(r"([.,?!])",r" \1 ",line[1])
    line[1]=re.sub(r"[0-9]",r" ",line[1])
    line[1]=re.sub(r'["]',r" ",line[1])
    line[1]=re.sub(r'[—]',r"",line[1])
    line[1]=re.sub(r'[„]',r"",line[1])
    line[1]=re.sub(r'[“]',r"",line[1])
    line[1]=re.sub(r'[–]',r"",line[1])
    line[1]=re.sub(r'[‘‚]',r"",line[1])
    line[1]=re.sub(r"[']",r" ",line[1])
    line[1]=re.sub(r"[:]",r" ",line[1])
    line[1]=re.sub(r'[" "]+'," ",line[1])
    line[1]=line[1].strip()
    line[1]="<SOS> "+line[1]+" <EOS>"

    proc_lines.append(line) 
  return pd.DataFrame.from_records(data=proc_lines,columns=['eng','ger']) 

In [72]:
df=process_text(lines)
df.head(10)
df[df['eng']=="<SOS> tom will be here soon . how soon ? <EOS>"]['ger']

154973    <SOS> tom wird bald hier sein .   was heißt ba...
Name: ger, dtype: object

In [57]:
df.tail(10)

,eng,ger
224410,"<SOS> as a prank , some students let three goa...",<SOS> als streich ließen einige schüler drei ...
224411,<SOS> the small crowd at hiroshima peace memor...,"<SOS> um . uhr vormittags , dem genauen zeitpu..."
224412,"<SOS> in today s world , we have to equip all ...",<SOS> in der heutigen welt müssen wir all uns...
224413,<SOS> death is something that we re often disc...,"<SOS> wir werden oft davon abgehalten , über ..."
224414,<SOS> at a moment when our economy is growing ...,"<SOS> in einem moment , in dem unsere wirtscha..."
224415,<SOS> even if some sentences by non-native spe...,<SOS> auch wenn sätze von nichtmuttersprachle...
224416,<SOS> if someone who doesn t know your backgro...,"<SOS> wenn jemand , der deine herkunft nicht k..."
224417,<SOS> if someone who doesn t know your backgro...,"<SOS> wenn jemand fremdes dir sagt , dass du d..."
224418,<SOS> if someone who doesn t know your backgro...,"<SOS> wenn jemand , der nicht weiß , woher man..."
224419,<SOS> doubtless there exists in this world pre...,<SOS> ohne zweifel findet sich auf dieser welt...


In [39]:
class Vocab_builder():
  def __init__(self,lang,series):
    self.lang=lang
    self.data=series
  def tokenize(self,line):
    return line.split(' ')
  def build_vocab(self):
    self.uniq_words=set()
    self.maxlen=0
    count=0
    self.num_list=[]
    for index,line in self.data.items():
      self.word_list=self.tokenize(line)
      self.maxlen=max(len(self.word_list),self.maxlen)
      for word in self.word_list:
        if(word not in self.uniq_words):
          self.uniq_words.add(word)
          self.num_list.append(count)
          count+=1
      
    self.vocab_list=list(self.uniq_words)
    print("Built vocabulary having {} elements".format(len(self.vocab_list)))
    print("Largest sentence length (with tags):{}".format(self.maxlen))
    return dict(zip(self.vocab_list,self.num_list)),dict(zip(self.num_list,self.vocab_list))






In [40]:
eng=Vocab_builder('eng',df['eng'])
ger=Vocab_builder('ger',df['ger'])


In [41]:
eng_w2i,eng_i2w=eng.build_vocab()

Built vocabulary having 19396 elements
Largest sentence length (with tags):112


In [42]:
eng_w2i

{'lacked': 0,
 'reasons': 1,
 'watering': 2,
 'inedible': 3,
 'blame': 4,
 'free-for-all': 5,
 'ounces': 6,
 'fueled': 7,
 'Pigs': 8,
 'shoulders': 9,
 'ongoing': 10,
 'doctorate': 11,
 'bothered': 12,
 'hacking': 13,
 'reflected': 14,
 '40': 15,
 'Air': 16,
 'hiding': 17,
 'smelled': 18,
 'dream': 19,
 'gist': 20,
 'coin': 21,
 're-election': 22,
 'beggar': 23,
 'breaks': 24,
 'Park': 25,
 '<SOS>': 26,
 'Major': 27,
 'Spain': 28,
 'swallowing': 29,
 "You'll": 30,
 'curtain': 31,
 'earlier': 32,
 'mixture': 33,
 'staple': 34,
 'twisting': 35,
 'executive': 36,
 'Stone': 37,
 'racing': 38,
 'officers': 39,
 'courthouse': 40,
 'Nikkei': 41,
 'erupt': 42,
 'defenses': 43,
 'shooting': 44,
 'herd': 45,
 'surprising': 46,
 'Maybe': 47,
 'overly': 48,
 'espresso': 49,
 'rotting': 50,
 'Lidya': 51,
 'stormy': 52,
 'buildings': 53,
 'generously': 54,
 'Drinking': 55,
 'Questioned': 56,
 'horizontal': 57,
 'Drinks': 58,
 'Nicholas': 59,
 'efficiently': 60,
 'smashed': 61,
 'Dental': 62,
 'Valen